In [1]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import RegexpTokenizer

import os
import sys

sys.path.append("../../data_preprocess/")

import pickle
import re
from utils import json_pretty_dump, word2VecContinueLearning, trainWord2VecModelType2, tokenizeData, convertWord2Vec, text_cleansing, parse_datetime, trainWord2VecModel
from sklearn.model_selection import train_test_split

seed = 42
np.random.seed(seed)

data_dir = "../processed/BGL_preprocessed"

params = {
    "log_file": "../../Drain_result/BGL_2.log_structured.csv",
    "template_file": "../../Drain_result/BGL_2.log_templates.csv",
    "test_ratio": 0.2,
    "train_anomaly_ratio": 0.0,
    "train_word2Vec": True
}

data_name = f'bgl_{params["train_anomaly_ratio"]}_tar'

data_dir = os.path.join(data_dir, data_name)
os.makedirs(data_dir, exist_ok=True)

In [2]:
struct_log = pd.read_csv(params["log_file"], engine="c", na_filter=False, memory_map=True)
struct_log['Label'] = struct_log['Label'].map(lambda x: int(x != "-"))

In [3]:
eventTemplateTokenTrain = []
eventTemplateTokenTest = []
eventVectors = []

In [4]:
train_data, test_data = train_test_split(struct_log, test_size=params["test_ratio"], random_state=42)
train_data = train_data.loc[train_data['Label'] == 0]

In [5]:
train_data = train_data.sort_values(by=['Time'])
test_data = test_data.sort_values(by=['Time'])

eventTemplateTokenTrain = tokenizeData(train_data)
eventTemplateTokenTest = tokenizeData(test_data)

In [6]:
if params["train_word2Vec"]:
    trainWord2VecModel(eventTemplateTokenTrain, 'word2vec_bgl')

start train word2Vec model. . . . .
finish train word2Vec model . . . . . ^^


In [7]:
model = Word2Vec.load("../../word2vec_bgl.model")
eventVectors = convertWord2Vec(eventTemplateTokenTrain, model)
train_data['EventTemplateIdentToken'] = eventTemplateTokenTrain
train_data['Vectors'] = eventVectors

In [8]:
eventVectors = convertWord2Vec(eventTemplateTokenTest, model)
test_data['EventTemplateIdentToken'] = eventTemplateTokenTest
test_data['Vectors'] = eventVectors

In [9]:
with open(os.path.join(data_dir, "session_train.pkl"), "wb") as fw:
    pickle.dump(train_data, fw)
with open(os.path.join(data_dir, "session_test.pkl"), "wb") as fw:
    pickle.dump(test_data, fw)
json_pretty_dump(params, os.path.join(data_dir, "data_desc.json"))